#BERT for sentiment analysis of customer reviews

In [1]:
%%writefile requirements.txt

transformers
torch
pandas
scikit-learn
streamlit


Writing requirements.txt


In [2]:
%%writefile sample_reviews.csv

text,label
"This movie was fantastic!",positive
"I hated the acting and the plot.",negative
"One of the best experiences I've had.",positive
"The film was boring and too long.",negative


Writing sample_reviews.csv


In [3]:
%%writefile predict.py

# model/predict.py
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import os

MODEL_NAME = 'nlptown/bert-base-multilingual-uncased-sentiment'

# Load model and tokenizer once
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME)

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    confidence, prediction = torch.max(probs, dim=1)
    sentiment = int(prediction.item()) + 1  # 1 to 5 star
    return sentiment, confidence.item()


Writing predict.py


In [4]:
%%writefile app.py

import streamlit as st
from model.predict import predict_sentiment

st.set_page_config(page_title="BERT Sentiment Classifier", layout="centered")
st.title("🧠 BERT Sentiment Analysis")
st.write("Enter a customer review below to analyze its sentiment.")

user_input = st.text_area("✍️ Review Text", height=200)

if st.button("Analyze Sentiment"):
    with st.spinner("Analyzing using BERT..."):
        rating, confidence = predict_sentiment(user_input)
        st.success(f"🌟 Predicted Rating: {rating} stars")
        st.info(f"Confidence: {confidence:.2f}")


Writing app.py
